In [3]:
import cv2
import pandas as pd
import numpy as np
import sys



sys.path.append("../src")

import make_camera_coordinates as mcc

get the rotation matricies for all points in C looking at all points in P

    INPUTS
    ------
    C: numpy array of shape (num camera points, 3)
    P: numpy array of shape (num target points, 3)

    OUTPUTS
    -------

    extrinsic_camera_matrix : numpy array of shape (num target points, num camera points, 3, 3)

    algorithm for look-at camera rotation matrix
    1. Compute L = p - C.
    2. Normalize L.
    3. Compute s = L x u. (cross product)
    4. Normalize s.
    5. Compute u_ = s x L.
    6. Then Extrinsic rotation matrix given by:

            s1,  s2,  s3
    R =     u_1, u_2, u_3
            -L1, -L2, -L3

    u is the y-axis -- vary this to get roll

    This takes advantage of broadcasting 
        -- see https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

In [379]:

# point we want to project
b = np.array([3,0,0,1])

# target point
p = np.array([-1.524 , -0.6096,  0.    ])

# camera center in world coordinates
C = np.array([ -6.096 ,   0.9144,  15.24  ])

# up vector
u = np.array([0,1,0], dtype='float32')


#inrinsic parameters

# physical sensor width and hiehgt in iphone in real life in m
F  = 4.11/1000. # focal length
W = 4.8/1000.
H = 3.6 / 1000.  
w = 1280. # width in pixels
h = 720 # ehight in pixels


fy=1229.; cy=360.; fx=1153.; cx=640.;
fy=4.11/1000.; cy=0; fx=4.11/1000.; cx=0;
fy=F*h/H; cy=0; fx=F*w/W; cx=0;

K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])

In [390]:
# rotation matrix
L = p - C
print "L after sub: %s" % L
L = L / np.linalg.norm(L)
print "L after norm: %s" % L
s = np.cross(L, u)
s = s / np.linalg.norm(s)
print "s after norm: %s" % s
u_ = np.cross(s, L)
print "u_: %s" % u_
R = np.vstack([s, u_, -L])
print "R before fiddle:\n %s" % R
    
# transformation vector
R[0] = -R[0]
R[2] = -R[2]
print "final R:\n%s" % R
t = -np.dot(R, C)
print "t: %s" % t

extrinsic_matrix = np.hstack([R, t.reshape(-1,1)])
print "extrinsic_matrix:\n %s" % extrinsic_matrix

L after sub: [  4.572  -1.524 -15.24 ]
L after norm: [ 0.28603878 -0.09534626 -0.95346259]
s after norm: [ 0.95782629 -0.          0.28734789]
u_: [ 0.02739755  0.99544417 -0.09132515]
R before fiddle:
 [[ 0.95782629 -0.          0.28734789]
 [ 0.02739755  0.99544417 -0.09132515]
 [-0.28603878  0.09534626  0.95346259]]
final R:
[[-0.95782629  0.         -0.28734789]
 [ 0.02739755  0.99544417 -0.09132515]
 [ 0.28603878 -0.09534626 -0.95346259]]
t: [ -1.45972726   0.64857662  16.36164686]
extrinsic_matrix:
 [[ -0.95782629   0.          -0.28734789  -1.45972726]
 [  0.02739755   0.99544417  -0.09132515   0.64857662]
 [  0.28603878  -0.09534626  -0.95346259  16.36164686]]


In [378]:
print "C vector: position of camera in world coordinates"
print C
print "*******"
print "t vector, position of world origin in camera coordinates"
print t
print "*******"
b_cam = np.dot(extrinsic_matrix, b)
print "B_cam: the location of the point in camera coordinates"
print b_cam
print "*******"
print "extrinsic_matrix is "
print  extrinsic_matrix
print "*******"
print "K is the camera matrix"
print K
print "*******"
P = np.dot(K, extrinsic_matrix)
print "P is: camera matrix"
print P
print "*******"
print "we are looking from:"
print C
print "*******"
print "the point we are projecting is:"
print b
print "we are looking at:"
print p
print "the projection into the camera is:"
print np.dot(P, b)

C vector: position of camera in world coordinates
[   0    0 -100]
*******
t vector, position of world origin in camera coordinates
[ -6.98291277  -0.          99.75589671]
*******
B_cam: the location of the point in camera coordinates
[ -3.99023587   0.          99.9653841 ]
*******
extrinsic_matrix is 
[[  9.97558967e-01  -0.00000000e+00  -6.98291277e-02  -6.98291277e+00]
 [  0.00000000e+00   1.00000000e+00  -0.00000000e+00  -0.00000000e+00]
 [  6.98291277e-02   0.00000000e+00   9.97558967e-01   9.97558967e+01]]
*******
K is the camera matrix
[[  1.09600000e+03   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   8.22000000e+02   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]
*******
P is: camera matrix
[[  1.09332463e+03   0.00000000e+00  -7.65327240e+01  -7.65327240e+03]
 [  0.00000000e+00   8.22000000e+02   0.00000000e+00   0.00000000e+00]
 [  6.98291277e-02   0.00000000e+00   9.97558967e-01   9.97558967e+01]]
*******
we are looking from:
[   0    0 -100

In [371]:
#using this wiki article: https://en.wikipedia.org/wiki/Pinhole_camera_model
# 4.11 comes from 4.11 mm focal length (in real world terms) of iphone 6 
#    -- see http://photo.stackexchange.com/questions/57560/so-my-iphone-6-camera-lens-is-as-wide-as-my-full-frame-35mm-dslr-lens

# TODO: add aspect ratio
f  = 4.11/1000.
W = 4.8/1000.
H = 3.6 / 1000.   # http://photoseek.com/2013/compare-digital-camera-sensor-sizes-full-frame-35mm-aps-c-micro-four-thirds-1-inch-type/
y = b_cam[0:2]*(f / b_cam[2])

# flip again because it will be on the other side
resolution =np.array((1280, 640))
w, h = resolution
print "THIS WORKS\n"
# actual length in meters
print "actual length in meters"
print y
print "\n divide by sensor width:"
print "\nlength in terms of percent of screen"
pct_scr = y / W
print pct_scr
print "\n multiply by number of pixels wide the image is"
print "\nlength in terms of pixels"
relative_pixels_from_center = pct_scr * resolution
print relative_pixels_from_center
#print - projection / sensor_width
print "\n add half of the image pixels wide"
print '\nour projection is:'
print relative_pixels_from_center + resolution/2

THIS WORKS

actual length in meters
[-0.00016406  0.        ]

 divide by sensor width:

length in terms of percent of screen
[-0.03417823  0.        ]

 multiply by number of pixels wide the image is

length in terms of pixels
[-43.74812893   0.        ]

 add half of the image pixels wide

our projection is:
[ 596.25187107  320.        ]


In [320]:
fx=1229

array([ -3.99023587,   0.        ,  99.9653841 ])

In [185]:
print intrinsic_matrix
print np.dot(intrinsic_matrix, wc)

[[1153    0  640]
 [   0 1229  360]
 [   0    0    1]]
[[ 55792.47547326]
 [ 35912.1228171 ]
 [    99.75589671]]


In [186]:
toScreen = np.array([[640, 0,0, 640],[0, -360,0, 360], [0, 0, 1, 0], [0, 0, 0,1]])
print toScreen

[[ 640    0    0  640]
 [   0 -360    0  360]
 [   0    0    1    0]
 [   0    0    0    1]]


In [187]:
# this seems to wrok
np.dot(toScreen, np.vstack([wc,[1]]))

array([[ -3.82906417e+03],
       [  3.60000000e+02],
       [  9.97558967e+01],
       [  1.00000000e+00]])

In [129]:
-1.19902647e+03

-1199.02647

In [81]:
np.linalg.det(R)

0.99999999999999989

In [149]:
def rotate_x(theta_x):
    r1 = [1,0,0,0]
    r2 = [0, np.cos(theta_x), -np.sin(theta_x), 0]
    
    r3 = [0, np.sin(theta_x), np.cos(theta_x), 0]
    
    r4 = [0,0,0,1]
    return np.array([r1,r2,r3,r4])

def rotate_y(theta_y):
    r1 = [np.cos(theta_y), 0, np.sin(theta_y), 0]
    r2 = [0, 1, 0, 0]
    
    r3 = [-np.sin(theta_y),0, np.cos(theta_y), 0]
    
    r4 = [0,0,0,1]
    return np.array([r1,r2,r3,r4])

def rotate_z(theta_z):
    r1 = [np.cos(theta_z), -np.sin(theta_z), 0, 0]
    r2 = [np.sin(theta_z), np.cos(theta_z), 0, 0 ]
    
    r3 = [0, 0, 1, 0]
    
    r4 = [0, 0, 0, 1]
    return np.array([r1,r2,r3,r4])

def rotate(p, thetas=(0,0,0), degrees=False, transpose=False):
    """ rotate x, then y, then z"""
    if degrees == True:
        thetas = [np.deg2rad(th) for th in thetas]
    X_rot_mat = rotate_x(thetas[0])
    Y_rot_mat = rotate_y(thetas[1])
    Z_rot_mat = rotate_z(thetas[2])
    if transpose:
        X_rot_mat = X_rot_mat.T
        Y_rot_mat = Y_rot_mat.T
        Z_rot_mat = Z_rot_mat.T
        
    rot_mat = np.dot(np.dot(Z_rot_mat, Y_rot_mat), X_rot_mat)
    return np.dot(rot_mat, p), rot_mat

    #return np.dot(np.dot(np.dot(rotate_x(thetas[0]), rotate_y(thetas[1])), rotate_z(thetas[2])), p)

def CtoT(C):
    """C is camera position, will return translation matrix"""
    
    return np.vstack([np.hstack([np.identity(3), C.reshape(-1,1)]), np.array([0,0,0,1])])


In [18]:
np.array([1,1,0,1])*rotate_z(np.deg2rad(45))


array([[ 0.70710678, -0.70710678,  0.        ,  0.        ],
       [ 0.70710678,  0.70710678,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [21]:
np.dot(rotate_y(np.deg2rad(45)), np.array([1,0,0,1]))


array([ 0.70710678,  0.        , -0.70710678,  1.        ])

In [121]:
p1, R = rotate(np.array([1,0,0,1]),thetas=(0,45, 45), degrees=True,transpose=False)

In [122]:
p1

array([ 0.5       ,  0.5       , -0.70710678,  1.        ])

In [123]:
R

array([[ 0.5       , -0.70710678,  0.5       ,  0.        ],
       [ 0.5       ,  0.70710678,  0.5       ,  0.        ],
       [-0.70710678,  0.        ,  0.70710678,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [150]:
C = np.array([0,0,5]) # position of camera

Tr = CtoT(C)
Tr

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  5.],
       [ 0.,  0.,  0.,  1.]])

In [151]:
T = np.dot(R, Tr)

In [152]:
T

array([[ 0.5       , -0.70710678,  0.5       ,  2.5       ],
       [ 0.5       ,  0.70710678,  0.5       ,  2.5       ],
       [-0.70710678,  0.        ,  0.70710678,  3.53553391],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [153]:
np.dot(T,np.array([1,0,0,1]))

array([ 3.        ,  3.        ,  2.82842712,  1.        ])

In [154]:
p1

array([ 0.5       ,  0.5       , -0.70710678,  1.        ])

In [155]:
np.dot(Tr, p1)

array([ 0.5       ,  0.5       ,  4.29289322,  1.        ])

In [156]:
np.dot(Tr, np.array([1,0,0,1]))

array([ 1.,  0.,  5.,  1.])